In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from datetime import datetime

In [2]:
%run Data_Cleaning_Utils.ipynb

In [3]:
data_path = './PM2.5/'
selected_years = [2017, 2018, 2019, 2020, 2021, 2022]
os.listdir(data_path)
data_name_lst = [name for name in os.listdir(data_path) if name.startswith('PM2.5')]
selected_files = []
for i in selected_years:
    for j in data_name_lst:
        if str(i) in j:
            selected_files.append(j)
save_path = './Final_Data/'

In [4]:
# Fill nan by their own record.
fill_nan_year = Fill_NaN_YearRecord(data_path, selected_years)
PM_data_0 = fill_nan_year.main()

# Fill nan by the data from same provide.
label_provide_df = made_provide_station(selected_files, data_path)
fill_nan_provide = Fill_NaN_Label(data_path, selected_years, label_provide_df)
PM_data_1 = fill_nan_provide.main(PM_data=PM_data_0)

# Fill nan by the data from same sector of thailand.
meteorology_label = meteorology_labeling()
label_df = made_sector_station(selected_files, data_path, meteorology_label)
fill_nan_provide = Fill_NaN_Label(data_path, selected_years, label_df)
# fill_nan_provide.save_name = 'FillByAll3'
PM_data_2 = fill_nan_provide.main(PM_data=PM_data_1)

# Fill nan by interpolation between themselves data
fill_nan_inter = Fill_NaN_Interpolation(data_path, selected_files)
fill_nan_inter.save_name = 'FillByAll4'
PM_data_3 = fill_nan_inter.main(PM_data=PM_data_2, save_path=save_path)

--------Before interpolating--------
2017 Num: 312 Failed
2018 Num: 3 Failed
2019 Num: 0 Pass
2020 Num: 0 Pass
2021 Num: 0 Pass
2022 Num: 0 Pass
-------------------------------------
--------After interpolating--------
2017 Num: 160 Failed
2018 Num: 0 Pass
2019 Num: 0 Pass
2020 Num: 0 Pass
2021 Num: 0 Pass
2022 Num: 0 Pass
-------------------------------------


In [5]:
PM_data_0['2017']

,Date,05T,59T,61T,50T,52T,53T,54T,08T,19T,...,77T,35T,36T,40T,75T,76T,44T,62T,63T,80T
0,2017_01_01,33.0,21.0,28.0,NaN,34.0,NaN,NaN,28.0,NaN,...,18.0,30.0,NaN,21.0,16.0,20.0,7.0,NaN,NaN,NaN
1,2017_01_02,28.0,19.0,27.0,NaN,32.0,NaN,NaN,26.0,NaN,...,15.0,NaN,NaN,11.0,5.0,18.0,9.0,NaN,NaN,NaN
2,2017_01_03,31.0,19.0,32.0,NaN,34.0,NaN,NaN,28.0,NaN,...,19.0,NaN,13.0,12.0,4.0,21.0,6.0,NaN,NaN,NaN
3,2017_01_04,37.0,21.0,34.0,NaN,37.0,NaN,NaN,28.0,NaN,...,21.0,19.0,NaN,10.0,2.0,18.0,5.0,NaN,NaN,NaN
4,2017_01_05,45.0,29.0,40.0,NaN,45.0,NaN,NaN,34.0,NaN,...,29.0,NaN,NaN,7.0,NaN,12.0,7.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359,2017_12_27,38.0,NaN,46.0,45.0,52.0,25.0,NaN,56.0,48.0,...,27.0,35.0,26.0,21.0,29.0,21.0,9.0,10.0,16.0,19.0
360,2017_12_28,18.0,21.0,26.0,29.0,37.0,18.0,NaN,30.0,30.0,...,23.0,16.0,10.0,6.0,NaN,9.0,8.0,7.0,17.0,10.0
361,2017_12_29,14.0,15.0,20.0,24.0,31.0,15.0,NaN,24.0,23.0,...,25.0,19.0,15.0,11.0,11.0,10.0,9.0,11.0,15.0,9.0
362,2017_12_30,14.0,22.0,22.0,24.0,31.0,16.0,NaN,26.0,24.0,...,25.0,20.0,17.0,12.0,12.0,14.0,9.0,12.0,17.0,8.0


In [6]:
PM_data_1['2017']

,Date,05T,59T,61T,50T,52T,53T,54T,08T,19T,...,77T,35T,36T,40T,75T,76T,44T,62T,63T,80T
0,2017_01_01,33.0,21.0,28.0,29.00,34.0,29.00,29.000000,28.0,28.0,...,18.0,30.0,30.0,21.0,16.0,20.0,7.0,NaN,NaN,NaN
1,2017_01_02,28.0,19.0,27.0,26.50,32.0,26.50,26.500000,26.0,26.0,...,15.0,NaN,NaN,11.0,5.0,18.0,9.0,NaN,NaN,NaN
2,2017_01_03,31.0,19.0,32.0,29.00,34.0,29.00,29.000000,28.0,28.0,...,19.0,13.0,13.0,12.0,4.0,21.0,6.0,NaN,NaN,NaN
3,2017_01_04,37.0,21.0,34.0,32.25,37.0,32.25,32.250000,28.0,28.0,...,21.0,19.0,19.0,10.0,2.0,18.0,5.0,NaN,NaN,NaN
4,2017_01_05,45.0,29.0,40.0,39.75,45.0,39.75,39.750000,34.0,34.0,...,29.0,NaN,NaN,7.0,NaN,12.0,7.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359,2017_12_27,38.0,41.2,46.0,45.00,52.0,25.00,41.200000,56.0,48.0,...,27.0,35.0,26.0,21.0,29.0,21.0,9.0,10.0,16.0,19.0
360,2017_12_28,18.0,21.0,26.0,29.00,37.0,18.00,24.833333,30.0,30.0,...,23.0,16.0,10.0,6.0,NaN,9.0,8.0,7.0,17.0,10.0
361,2017_12_29,14.0,15.0,20.0,24.00,31.0,15.00,19.833333,24.0,23.0,...,25.0,19.0,15.0,11.0,11.0,10.0,9.0,11.0,15.0,9.0
362,2017_12_30,14.0,22.0,22.0,24.00,31.0,16.00,21.500000,26.0,24.0,...,25.0,20.0,17.0,12.0,12.0,14.0,9.0,12.0,17.0,8.0


In [7]:
PM_data_2['2017']

,Date,05T,59T,61T,50T,52T,53T,54T,08T,19T,...,77T,35T,36T,40T,75T,76T,44T,62T,63T,80T
0,2017_01_01,33.0,21.0,28.0,29.00,34.0,29.00,29.000000,28.0,28.0,...,18.0,30.000000,30.000000,21.0,16.00,20.0,7.0,7.0,7.0,NaN
1,2017_01_02,28.0,19.0,27.0,26.50,32.0,26.50,26.500000,26.0,26.0,...,15.0,11.333333,11.333333,11.0,5.00,18.0,9.0,9.0,9.0,NaN
2,2017_01_03,31.0,19.0,32.0,29.00,34.0,29.00,29.000000,28.0,28.0,...,19.0,13.000000,13.000000,12.0,4.00,21.0,6.0,6.0,6.0,NaN
3,2017_01_04,37.0,21.0,34.0,32.25,37.0,32.25,32.250000,28.0,28.0,...,21.0,19.000000,19.000000,10.0,2.00,18.0,5.0,5.0,5.0,NaN
4,2017_01_05,45.0,29.0,40.0,39.75,45.0,39.75,39.750000,34.0,34.0,...,29.0,9.500000,9.500000,7.0,9.50,12.0,7.0,7.0,7.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359,2017_12_27,38.0,41.2,46.0,45.00,52.0,25.00,41.200000,56.0,48.0,...,27.0,35.000000,26.000000,21.0,29.00,21.0,9.0,10.0,16.0,19.0
360,2017_12_28,18.0,21.0,26.0,29.00,37.0,18.00,24.833333,30.0,30.0,...,23.0,16.000000,10.000000,6.0,10.25,9.0,8.0,7.0,17.0,10.0
361,2017_12_29,14.0,15.0,20.0,24.00,31.0,15.00,19.833333,24.0,23.0,...,25.0,19.000000,15.000000,11.0,11.00,10.0,9.0,11.0,15.0,9.0
362,2017_12_30,14.0,22.0,22.0,24.00,31.0,16.00,21.500000,26.0,24.0,...,25.0,20.000000,17.000000,12.0,12.00,14.0,9.0,12.0,17.0,8.0


In [8]:
PM_data_3['2017']

,Date,05T,59T,61T,50T,52T,53T,54T,08T,19T,...,77T,35T,36T,40T,75T,76T,44T,62T,63T,80T
0,2017_01_01,33.0,21.0,28.0,29.00,34.0,29.00,29.000000,28.0,28.0,...,18.0,30.000000,30.000000,21.0,16.00,20.0,7.0,7.0,7.0,NaN
1,2017_01_02,28.0,19.0,27.0,26.50,32.0,26.50,26.500000,26.0,26.0,...,15.0,11.333333,11.333333,11.0,5.00,18.0,9.0,9.0,9.0,NaN
2,2017_01_03,31.0,19.0,32.0,29.00,34.0,29.00,29.000000,28.0,28.0,...,19.0,13.000000,13.000000,12.0,4.00,21.0,6.0,6.0,6.0,NaN
3,2017_01_04,37.0,21.0,34.0,32.25,37.0,32.25,32.250000,28.0,28.0,...,21.0,19.000000,19.000000,10.0,2.00,18.0,5.0,5.0,5.0,NaN
4,2017_01_05,45.0,29.0,40.0,39.75,45.0,39.75,39.750000,34.0,34.0,...,29.0,9.500000,9.500000,7.0,9.50,12.0,7.0,7.0,7.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359,2017_12_27,38.0,41.2,46.0,45.00,52.0,25.00,41.200000,56.0,48.0,...,27.0,35.000000,26.000000,21.0,29.00,21.0,9.0,10.0,16.0,19.0
360,2017_12_28,18.0,21.0,26.0,29.00,37.0,18.00,24.833333,30.0,30.0,...,23.0,16.000000,10.000000,6.0,10.25,9.0,8.0,7.0,17.0,10.0
361,2017_12_29,14.0,15.0,20.0,24.00,31.0,15.00,19.833333,24.0,23.0,...,25.0,19.000000,15.000000,11.0,11.00,10.0,9.0,11.0,15.0,9.0
362,2017_12_30,14.0,22.0,22.0,24.00,31.0,16.00,21.500000,26.0,24.0,...,25.0,20.000000,17.000000,12.0,12.00,14.0,9.0,12.0,17.0,8.0
